In [231]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import numpy as np


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [232]:
df = pd.read_csv('spam.csv',sep='\t',names = ['label','message'])

In [233]:
df.head(5)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [234]:
df.shape

(5572, 2)

In [235]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()

In [236]:
corpus = []
for i in range(0,len(df)):
    message = re.sub('[^a-zA-Z]',' ',df['message'][i])
    message = message.lower()
    message = message.split()
    message = [ps.stem(word) for word in message if not word in stopwords.words('english')]
    message = ' '.join(message)
    corpus.append(message)

In [237]:
df['label'].unique()

array(['ham', 'spam'], dtype=object)

In [238]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [239]:
y = pd.get_dummies(df['label'],drop_first = True).to_numpy()

In [240]:
y = y.reshape(1,-1)[0]

In [241]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer(max_features=200)
Tfidf = TfidfVectorizer(max_features=200)

In [242]:
X_cv = cv.fit_transform(corpus).toarray()
Tfidf.fit(corpus)
X_Tfidf = Tfidf.transform(corpus).toarray()

In [243]:
X_cv.shape

(5572, 200)

In [244]:
X_Tfidf.shape

(5572, 200)

In [245]:
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

In [246]:
from sklearn.model_selection import train_test_split
X_cv_train,X_cv_test,y_cv_train,y_cv_test = train_test_split(X_cv,y,test_size = 0.2,random_state = 42)

print('bag of words:',X_cv_train.shape,X_cv_test.shape,y_cv_train.shape,y_cv_test.shape)

X_Tfidf_train, X_Tfidf_test, y_Tfidf_train, y_Tfidf_test = train_test_split(X_Tfidf,y,test_size = 0.2,random_state = 42)

print('Tf-Idf:',X_Tfidf_train.shape, X_Tfidf_test.shape, y_Tfidf_train.shape, y_Tfidf_test.shape)

bag of words: (4457, 200) (1115, 200) (4457,) (1115,)
Tf-Idf: (4457, 200) (1115, 200) (4457,) (1115,)


In [247]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
model_1_cv = MultinomialNB()
model_1_cv.fit(X_cv_train,y_cv_train)
y_pred_cv = model_1_cv.predict(X_cv_test)
print('Bag_of_words =\n',confusion_matrix(y_pred_cv,y_cv_test))

model_1_Tfidf = MultinomialNB()
model_1_Tfidf.fit(X_Tfidf_train,y_Tfidf_train)
y_pred_Tfidf = model_1_Tfidf.predict(X_Tfidf_test)
print('Tf_Idf = \n',confusion_matrix(y_pred_Tfidf,y_Tfidf_test))


Bag_of_words =
 [[950  15]
 [ 16 134]]
Tf_Idf = 
 [[962  23]
 [  4 126]]


In [248]:
import tensorflow as tf


In [249]:
model_2_cv = tf.keras.Sequential(
    [
     tf.keras.layers.Dense(10,activation = 'relu'),
     tf.keras.layers.Dense(10,activation = 'relu'),
     tf.keras.layers.Dense(10,activation = 'relu'),
     tf.keras.layers.Dense(1,activation = 'sigmoid')
    ]
)
model_2_cv.compile(loss = tf.keras.losses.binary_crossentropy,metrics = ['accuracy'],optimizer = tf.keras.optimizers.Adam())
model_2_cv.fit(X_cv_train,y_cv_train,epochs= 15,verbose = 0)
y_pred_cv = model_2_cv.predict(X_cv_test)
y_pred_cv = (y_pred_cv > 0.5)
print('Bag_of_words =\n',confusion_matrix(y_pred_cv,y_cv_test))

model_2_Tfidf = tf.keras.Sequential(
    [
     tf.keras.layers.Dense(10,activation = 'relu'),
     tf.keras.layers.Dense(10,activation = 'relu'),
     tf.keras.layers.Dense(10,activation = 'relu'),
     tf.keras.layers.Dense(1,activation = 'sigmoid')
    ]
)
model_2_Tfidf.compile(loss = tf.keras.losses.binary_crossentropy,metrics = ['accuracy'],optimizer = tf.keras.optimizers.Adam())
model_2_Tfidf.fit(X_Tfidf_train,y_Tfidf_train,epochs= 15,verbose = 0)
y_pred_Tfidf = model_2_Tfidf.predict(X_Tfidf_test)
y_pred_Tfidf = (y_pred_Tfidf>0.5)
print('Tf_Idf = \n',confusion_matrix(y_pred_Tfidf,y_Tfidf_test))

Bag_of_words =
 [[955  12]
 [ 11 137]]
Tf_Idf = 
 [[956  12]
 [ 10 137]]
